# pvData Overview

# An aggregate data type

Suppose want to encode a statistical summary of N floating point values.
Let’s say we want
1.	A mean (or some other sort average)
2.	The number of samples
3.	The standard deviation (or some other measure of spread)
4.	The maximum value
5.	The minimum value

We could encode this as structure consisting of an ordered set of fields.  
Each field has a name and a type.  
Let’s say:  
1.	A 64-bit floating point field for the average. Let’s call this type “double”. For the name of the field, we could choose “average”, but let’s call it value.
2.	A 64-bit integer field for the number of samples. Let’s call the type “long” and name the field “N”.
3.	A double field for the spread. Let’s call it “dispersion”
4.	A double field “max”
5.	A double field “min”



We can write a description of our type as follows:  
```java
    structure
        double value
        long N
        double dispersion
        double max
        double min
```

Let’s say we have 1000 samples with a mean of 2.0, a standard deviation of 1.0 and a minimum of 0.1 and a maximum of 4.4. We could our encoding of the statistical summary as:
 
```java
    structure
        double value       2.0
        long N            1000
        double dispersion  1.0 
        double max         4.4
        double min         0.1
```
This system of encoding data in EPICS V4 is called pvData and this way of writing pvData structures is called the pvData meta language.

Some jargon:
There are two types of objects in pvData: *Introspection objects* and *data objects*.

The first type of structure above, which describes the type of a piece of structured data, is an example of an *introspection object*.


The second structure, which carries values, is called a *data object*.
Structure data objects have an *introspection object* and some of the fields can carry data (as in the case above).

The values of the fields are referred to as the *value data* of the fields. The value data of a structure’s fields are called the *value data* of the structure.

The introspection object corresponding to a data object is referred to as its *introspection data* or its *introspection type*.

We can additionally give the type of the structure a name, called the type ID which helps identify the structure. The default type ID for a structure is **structure**.  
The above structure is an example of a standard structure (a Normative Type) called **NTAggregate**. So let’s give it the type ID `NTAggregate`.

When we write a structure we typically write it with the type ID:
```java
    NTAggregate
    double value
    long N
    double dispersion
    double max
    double min
```

and the data object above as
```java
    NTAggregate
    double value       2.0
    long N            1000
    double dispersion  1.0 
    double max         4.4
    double min         0.1
```

# Structure subfields


Now suppose we want to add timestamps for the first and last sample in the data. We could encode the timestamp using the structure:
```java
    time_t
        long secondsPastEpoch
        int nanoseconds
        int userTag
```
where `int` denotes a 32-bit signed integer

We could then add structure subfields to our NTAggregate:
```java
    NTAggregate
        double value
        long N
        double dispersion
        double max
        double min
        time_t firstTimeStamp
            long secondsPastEpoch
            int nanoseconds
            int userTag
        time_t lastTimeStamp
            long secondsPastEpoch
            int nanoseconds
            int userTag
```

The two timestamp fields are fields of the top-level structure as well as the previous scalar fields.

Their introspection type is the `time_t` structure described above. These fields are themselves structures with fields secondsPastEpoch, nanoseconds and userTag.  

The indentation rules are similar to the indentation rules in Python.

An example of a corresponding data object would be: 
```java
    NTAggregate
        double value       2.0
        long N            1000
        double dispersion  1.0 
        double max         4.4
        double min         0.1
        time_t firstTimeStamp
            long secondsPastEpoch 1460589140
            int nanoseconds         11436967
            int userTag                    0
        time_t lastTimeStamp
            long secondsPastEpoch 1460589149
            int nanoseconds        889605397
            int userTag                    0
```

<stop here>

pvData 
 
In general a pvData structure consists of
0 or more fields
A type ID
A data object